In [19]:
# https://docs.google.com/document/d/1e9oakyEftdHp4zGz2F0WP-_X7GEzEKYGaPvu1SguuCE/edit

import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
from datetime import date as dt
from datetime import timedelta as td



# https://pypi.org/project/yfinance/#description
import yfinance as yf

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

# Transformations:

### Transform 1: Group by industry

Calculate:
* Market value per industry (dollar amount) for the day
* % change of market value from previous day for each industry

(market value = adjusted close * volume)

In [2]:
portfolio2 = ['MSFT', 'AAPL', 'IBM', 'WMT', 'SHOP', 'LWLG', 'ALB', 'LYV', 'GOOGL', 'TTGT', 'TSLA', 'GME', 'AMZN', 'TGT', 'COST', 'COKE','TPL', 'BX', 'MORN', 'CBRE', 
            'NVDA', 'AMD', 'NEE']
portfolio = ['MSFT', 'AAPL'] # reduced portfolio to make python quicker (pre prod)

### Pull industry for each ticker (for eventual aggregation) as well as current market cap (updates daily)

In [3]:
%%time

# today's date
today = dt.today().strftime("%d/%m/%Y")

# hash the entirety of the api call data for each stock
company_info = {i: yf.Ticker(i).info for i in portfolio}

# pull the respective data we'd like from the hash (instead of calling the api twice for each stock)
industry_cap_dict = {i: [today, company_info[i]['sector'], company_info[i]['marketCap']] for i in portfolio}
    

industry_cap_dict

CPU times: user 768 ms, sys: 76 ms, total: 844 ms
Wall time: 8.25 s


{'MSFT': ['25/02/2022', 'Technology', 2228894302208],
 'AAPL': ['25/02/2022', 'Technology', 2690253062144]}

In [4]:
# transform dictionary into dataframe
industry_cap_df = pd.DataFrame.from_dict(industry_cap_dict,orient='index')

# add the stock tickers, currently in index, to a column named 'company'
industry_cap_df.insert(1, 'company', industry_cap_df.index)

# move the 'date' column to the index
industry_cap_df.index = industry_cap_df.iloc[:,0]

# drop the 'date' column
industry_cap_df.drop(industry_cap_df.columns[0], axis=1, inplace=True)

# remove the title for the index column
industry_cap_df.index.name = None

# rename the remaining columns
pd.DataFrame(industry_cap_df).rename(columns={1: 'sector', 2: 'market_cap'}, inplace=True)

# Data to be used for metrics (market cap by industry)

In [5]:
industry_agg = industry_cap_df.groupby(['sector']).sum()
industry_agg

,market_cap
sector,
Technology,4919147364352


# Data to be used for notification purposes

In [58]:
# set date to yesterday
yesterday = datetime.now() - td(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')



financial_info_df = yf.download(portfolio, yesterday, group_by='Ticker')
financial_info_df = financial_info_df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1) # Trenton McKinney, https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801
financial_info_df = financial_info_df[['Ticker','Adj Close']]
financial_info_df

[*********************100%***********************]  2 of 2 completed


,Ticker,Adj Close
Date,,
2022-02-24,AAPL,162.740005
2022-02-24,MSFT,294.589996
2022-02-25,AAPL,164.850006
2022-02-25,MSFT,297.309998


In [64]:
a = financial_info_df.sort_values(by=['Ticker'])
a

,Ticker,Adj Close
Date,,
2022-02-24,AAPL,162.740005
2022-02-25,AAPL,164.850006
2022-02-24,MSFT,294.589996
2022-02-25,MSFT,297.309998


In [84]:
%%time
#create unique list of names
UniqueNames = portfolio

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in UniqueNames}

for key in DataFrameDict.keys():
    DataFrameDict[key] = financial_info_df[:][financial_info_df.Ticker == key]

CPU times: user 1.27 ms, sys: 0 ns, total: 1.27 ms
Wall time: 1.25 ms


In [90]:
for stock in portfolio:
    DataFrameDict[stock]['percent_change'] = DataFrameDict[stock]['Adj Close'].pct_change()
    
    # set notification for threshold
    # if greater than 1%
    if DataFrameDict[stock]['percent_change'][-1] > 0.01:
        print(today, stock, DataFrameDict[stock]['percent_change'][-1])

25/02/2022 AAPL 0.012965469700934706
